In [1]:
# original code:
# https://github.com/asteroid-team/asteroid/blob/master/notebooks/00_GettingStarted.ipynb

# Asteroid is based on PyTorch and PyTorch-Lightning.
from torch import optim
from pytorch_lightning import Trainer

# We train the same model architecture that we used for inference above.
from asteroid.models import ConvTasNet

# In this example we use Permutation Invariant Training (PIT) and the SI-SDR loss.
from asteroid.losses import pairwise_neg_sisdr, PITLossWrapper

# MiniLibriMix is a tiny version of LibriMix (https://github.com/JorisCos/LibriMix),
# which is a free speech separation dataset.
from asteroid.data import LibriMix

# Asteroid's System is a convenience wrapper for PyTorch-Lightning.
from asteroid.engine import System

from custom_dataset_sandbox import CustomMixture

from torch.utils.data import DataLoader



In [2]:
# This will automatically download MiniLibriMix from Zenodo on the first run.
# train_loader, val_loader = LibriMix.loaders_from_mini(task="sep_clean", batch_size=16)

# train_loader, val_loader = LibriMix.mini_from_download(task="sep_clean")#, {"num_workers": 8})
csv_path = "C:/Users/ZaknafeinII/Desktop/University_of_Iowa/4_Senior/Spring/ECE4890_SD/voiceseperation/ML_data_preparation/MiniLibriMix/metadata"
train_loader = DataLoader(CustomMixture(csv_path, "example_train"))
val_loader = DataLoader(CustomMixture(csv_path, "example_val"))

# train_loader = LibriMix("C:/Users/ZaknafeinII/Desktop/University_of_Iowa/4_Senior/Spring/ECE4890_SD/voiceseperation/ML_data_preparation/MiniLibriMix/metadata/test_libri/", task="sep_clean", sample_rate=16000, n_src=2, segment=3, return_id=False)
# val_loader = LibriMix("C:/Users/ZaknafeinII/Desktop/University_of_Iowa/4_Senior/Spring/ECE4890_SD/voiceseperation/ML_data_preparation/MiniLibriMix/metadata/test_libri/", task="sep_clean", sample_rate=16000, n_src=2, segment=3, return_id=False)
# Tell DPRNN that we want to separate to 2 sources.
model = ConvTasNet(n_src=2)

# PITLossWrapper works with any loss function.
loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

optimizer = optim.Adam(model.parameters(), lr=1e-3)

system = System(model, optimizer, loss, train_loader, val_loader)

In [3]:
# # from torch import cuda
# # import torch
# # print(cuda.device_count())
# # print(torch.version.cuda)
# # print(torch.cuda.is_available())

# # # print(type(model))

# print(type(train_loader))
# print(type(val_loader))

# train_loader.__getitem__(0)
# val_loader.__getitem__(0)



In [4]:
# Train for 1 epoch using a single GPU. If you're running this on Google Colab,
# be sure to select a GPU runtime (Runtime → Change runtime type → Hardware accelarator).
trainer = Trainer(max_epochs=1, gpus=0, auto_select_gpus=False)
trainer = Trainer()
# trainer = Trainer()
trainer.fit(system)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type           | Params
---------------------------------------------
0 | model     | ConvTasNet     | 5.1 M 
1 | loss_func | PITLossWrapper | 0     
---------------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.202    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\ZaknafeinII\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


IN __getitem__()
READ METADATA COMPLETE
READ MIXTURE COMPLETE
mixture.shape: (121920,)
sources.shape: (2, 121920)
IN __getitem__()
READ METADATA COMPLETE
READ MIXTURE COMPLETE
mixture.shape: (53360,)
sources.shape: (2, 53360)


C:\Users\ZaknafeinII\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\ZaknafeinII\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:326: UserWarning: The number of training samples (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

IN __getitem__()
READ METADATA COMPLETE
READ MIXTURE COMPLETE
mixture.shape: (121920,)
sources.shape: (2, 121920)
IN __getitem__()
READ METADATA COMPLETE
READ MIXTURE COMPLETE
mixture.shape: (53360,)
sources.shape: (2, 53360)


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 31209472 bytes.